In [3]:
import pandas as pd

# Cargar los archivos
df_alquiler = pd.read_csv("C:/Users/pablo/proyecto/comparador/alquiler_con_tipos_detectados.csv")
df_detalles = pd.read_csv("C:/Users/pablo/proyecto/comparador/alquiler_detalles_scrapeados.csv")

# Asegurarse de que coincidan los nombres de columna
df_detalles.rename(columns={'url': 'link'}, inplace=True)

# Unir ambos datasets por la columna 'link'
df_alquiler_unido = pd.merge(df_alquiler, df_detalles, on='link', how='left')

# Guardar el archivo combinado (opcional)
df_alquiler_unido.to_csv("C:/Users/pablo/proyecto/comparador/alquiler_unificado_final.csv", index=False)

# Verificar columnas y registros resultantes
print("✅ Archivos unidos. Columnas totales:", df_alquiler_unido.shape[1])
print("Registros totales:", df_alquiler_unido.shape[0])


✅ Archivos unidos. Columnas totales: 24
Registros totales: 1477


In [8]:
print(df_alquiler.columns.tolist())

['titulo', 'ubicacion', 'precio', 'habitaciones', 'baños', 'metros', 'superficie_construida', 'superficie_util', 'link', 'url', 'lat', 'lon', 'es_piso', 'es_casa', 'es_atico', 'es_estudio', 'es_apartamento', 'es_duplex', 'es_chalet', 'es_finca', 'es_loft']


In [10]:
# Detectar y renombrar columnas que empiezan por 'es_'
nuevos_nombres = {
    col: col.replace('es_', '') 
    for col in df_alquiler_unido.columns 
    if col.startswith('es_')
}

# Aplicar renombrado
df_alquiler_unido.rename(columns=nuevos_nombres, inplace=True)

# Ver columnas renombradas
print("✅ Columnas actualizadas:")
print(df_alquiler_unido.columns.tolist())


✅ Columnas actualizadas:
['titulo', 'ubicacion', 'precio', 'habitaciones', 'baños', 'metros', 'superficie_construida', 'superficie_util', 'link', 'url', 'lat', 'lon', 'piso', 'casa', 'atico', 'estudio', 'apartamento', 'duplex', 'chalet', 'finca', 'loft', 'conservacion', 'precio_m2', 'certificado_energetico']


In [13]:
# Convertir 'precio_m2' a tipo numérico (float)
df_alquiler_unido['precio_m2'] = (
    df_alquiler_unido['precio_m2']
    .astype(str)
    .str.replace('.', '', regex=False)
    .str.replace(' €/m²', '', regex=False)
    .str.strip()
)

# Convertir a número y forzar NaN donde no se pueda
df_alquiler_unido['precio_m2'] = pd.to_numeric(df_alquiler_unido['precio_m2'], errors='coerce')

# Ver cuántos valores nulos quedan
print("Valores nulos en precio_m2:", df_alquiler_unido['precio_m2'].isnull().sum())


Valores nulos en precio_m2: 654


In [15]:
# Eliminar registros sin precio_m2
df_alquiler_unido = df_alquiler_unido.dropna(subset=['precio_m2'])

# Ver cuántos registros quedan
print("Registros tras eliminar precio_m2 nulo:", df_alquiler_unido.shape[0])



Registros tras eliminar precio_m2 nulo: 823


In [17]:
# Eliminar registros sin conservacion
df_alquiler_unido = df_alquiler_unido.dropna(subset=['conservacion'])

# Ver cuántos registros quedan
print("Registros tras eliminar conservacion nulo:", df_alquiler_unido.shape[0])


Registros tras eliminar conservacion nulo: 556


In [19]:
# Función para eliminar outliers con IQR
def eliminar_outliers_iqr(df, columnas):
    for col in columnas:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        df = df[(df[col] >= limite_inferior) & (df[col] <= limite_superior)]
    return df

# Aplicar la función a las columnas clave
columnas_outliers = ['precio', 'metros', 'precio_m2']
df_alquiler_unido = eliminar_outliers_iqr(df_alquiler_unido, columnas_outliers)

# Ver cuántos registros quedan
print("Registros tras eliminar outliers:", df_alquiler_unido.shape[0])


Registros tras eliminar outliers: 505


In [21]:
# Eliminar registros con menos de 25 metros cuadrados
df_alquiler_unido = df_alquiler_unido[df_alquiler_unido['metros'] >= 25]

# Reiniciar índices
df_alquiler_unido = df_alquiler_unido.reset_index(drop=True)

# Ver cuántos registros quedan
print("✅ Registros tras eliminar inmuebles con < 25 m²:", df_alquiler_unido.shape[0])


✅ Registros tras eliminar inmuebles con < 25 m²: 502


In [23]:
# Eliminar columna completamente vacía
df_alquiler_unido.drop(columns=['certificado_energetico'], inplace=True)

# Eliminar filas con valores nulos residuales (1 fila en lat/lon/sup_construida)
df_alquiler_unido = df_alquiler_unido.dropna(subset=['lat', 'lon', 'superficie_construida'])

# Reiniciar índice
df_alquiler_unido = df_alquiler_unido.reset_index(drop=True)

# Mostrar resultado final
print("✅ Dataset listo. Registros finales:", df_alquiler_unido.shape[0])


✅ Dataset listo. Registros finales: 501


In [27]:
# Eliminar la columna 'superficie_util'
df_alquiler_unido.drop(columns=['superficie_util'], inplace=True)

# Confirmar columnas actuales
print("✅ Columna 'superficie_util' eliminada.")
print("Columnas actuales:", df_alquiler_unido.columns.tolist())


✅ Columna 'superficie_util' eliminada.
Columnas actuales: ['titulo', 'ubicacion', 'precio', 'habitaciones', 'baños', 'metros', 'superficie_construida', 'link', 'url', 'lat', 'lon', 'piso', 'casa', 'atico', 'estudio', 'apartamento', 'duplex', 'chalet', 'finca', 'loft', 'conservacion', 'precio_m2']


In [33]:
# Guardar el DataFrame limpio de alquiler
df_alquiler_unido.to_csv("C:/Users/pablo/proyecto/comparador/alquiler_unificado_final.csv", index=False)

print("✅ Archivo guardado como alquiler_unificado_final.csv")



✅ Archivo guardado como alquiler_unificado_final.csv
